<a href="https://colab.research.google.com/github/johanhoffman/DD2363_VT21/blob/Diracturing/Lab_2/Diracturing_lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 2: **
**Julius Andersson**

# **Abstract**

#**About the code**

A short statement on who is the author of the file, and if the code is distributed under a certain license. 

In [15]:
"""This program is a template for lab reports in the course"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2020 Johan Hoffman (jhoffman@kth.se)

# This file is part of the course DD2365 Advanced Computation in Fluid Mechanics
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This template is maintained by Johan Hoffman
# Please report problems to jhoffman@kth.se

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

To have access to the neccessary modules you have to run this cell. If you need additional modules, this is where you add them. 

In [16]:
# Load neccessary modules.
from google.colab import files

import time
import numpy as np

#try:
#    from dolfin import *; from mshr import *
#except ImportError as e:
#    !apt-get install -y -qq software-properties-common 
#    !add-apt-repository -y ppa:fenics-packages/fenics
#    !apt-get update -qq
#    !apt install -y --no-install-recommends fenics
#    from dolfin import *; from mshr import *
    
#import dolfin.common.plotting as fenicsplot

from matplotlib import pyplot as plt
from matplotlib import tri
from matplotlib import axes
from mpl_toolkits.mplot3d import Axes3D

# **Introduction**
Not much to say

# **Method**

1) Sparse matrix-vector product. It follows algorithm 5.9 presented in the lecture notes.

In [90]:
def Sparse_Ax(A,x):
  b=[]
  for i in range(len(x)):
    tmp=0
    for j in range(A.row_ptr[i],A.row_ptr[i+1]):
      tmp+=A.val[j]*x[A.col_idx[j]]
    b.append(tmp)
  return b

class A:
  def __init__(self,A, x):
    self.A = A
    self.x = x
    self.val=[]
    self.col_idx=[]
    self.row_ptr=[]

  def init(self):
    next=0
    print=True
    for i in range(len(self.A)):
      for j in range(len(self.A[0])):
        if(self.A[i][j]!=0):
          if(print):
            self.row_ptr.append(next)
            print=False
          next+=1
          self.val.append(self.A[i][j])
          self.col_idx.append(j)
        if(j==len(self.A[0])-1 and print==True):  ###
          self.row_ptr.append(self.row_ptr[-1])
      print=True
    self.row_ptr.append(len(self.val))
      

######Test##########
Arr=[[1,0,0],[0,1,0],[0,0,1]]
x=[23,35,40]

Arr2=[[3,2,0,2,0,0],[0,2,1,0,0,0],[0,0,1,0,0,0],[0,0,3,2,0,0],[0,0,0,0,1,0],[0,0,0,0,2,3]]
x2=[2,3,2,3,2,3]

tmp=A(Arr,x)
tmp2=A(Arr2,x2)
tmp.init()
tmp2.init()

if(not (Sparse_Ax(tmp,tmp.x)==np.matmul(Arr,x)).all()):
    print("err")

if(not (Sparse_Ax(tmp2,tmp2.x)==np.matmul(Arr2,x2)).all()):
    print("err")


2) QR factorization. It is the modified gram schmidt algorithm from algorithm 5.3 in the lectures notes. 

In [94]:
def QR(A):
  Q=np.zeros(A.shape)
  R=np.zeros(A.shape)

  for j in range(A.shape[0]):
    v=A[:,j]
    for i in range(j):
      R[i,j]=np.dot(Q[:,i], v)
      v=v-R[i,j]*Q[:,i]
    R[j,j]=np.dot(v,v)**(1/2)
    Q[:,j]=v/R[j,j]         #
  return Q,R




#####Test########
A=np.array([[2,-1,5],[-1,2,4],[10,3,5]])
A2=np.array([[2,-1],[-1,2]])

Q,R=QR(A)
print(np.allclose(R, np.triu(R))) # check if upper triangular
print(np.linalg.norm(np.matmul(np.transpose(Q),Q)-np.identity(Q.shape[0])))      ## linalg.norm uses frebenius norm
print(np.linalg.norm(np.matmul(Q,R)-A))


Q,R=QR(A2)
print(np.allclose(R, np.triu(R))) # check if upper triangular
print(np.linalg.norm(np.matmul(np.transpose(Q),Q)-np.identity(Q.shape[0])))
print(np.linalg.norm(np.matmul(Q,R)-A2))





True
1.890129964886933e-16
4.965068306494546e-16
True
2.6901577681355055e-16
0.0


3) Direct solver Ax=b. From 2) we know how to solve $A=QR$. To solve $Ax=b$ we can rewrite it as $QRx=b$. $Rx=Q^{-1}b=b'$. We can solve $Rx=b'$ with backward substitution as presented in algorithm 5.2.

In [96]:
def backward(U,b):
  x=np.zeros(len(b))
  n=len(b)
  x[n-1]=b[n-1]/U[n-1,n-1]
  for i in reversed(range(n-1)):
    sum=0
    for j in range(i+1,n):
      sum+=U[i,j]*x[j]
    x[i]=(b[i]-sum)/U[i,i]
  return x

def directSolver(A,b):
  Q,R=QR(A)
  bprime=np.matmul(np.transpose(Q),b)
  return backward(R,bprime)


#####TEST##########

A=np.array([[2,3],[1,3]])
b=np.array([5,4])
x=directSolver(A,b)

print(np.dot(np.matmul(A,x)-b,np.matmul(A,x)-b)**(1/2))
y=directSolver(A,b)
print(np.dot(x-y,x-y))

A=np.array([[2,3,5],[1,3,6],[3,6,1]])
b=np.array([5,4,9])
x=directSolver(A,b)

print(np.dot(np.matmul(A,x)-b,np.matmul(A,x)-b)**(1/2))
y=directSolver(A,b)
print(np.dot(x-y,x-y))


1.9860273225978185e-15
0.0
8.103867617357112e-15
0.0


# **Results**

# **Discussion**

# **APPENDIX**